In [27]:
from langgraph.graph import StateGraph, START,END 
from langchain_google_genai import ChatGoogleGenerativeAI 
import os 
from typing import TypedDict, Annotated, Literal 
from langchain_core.messages import BaseMessage, HumanMessage 
from dotenv import load_dotenv 
from pydantic import BaseModel,Field 
from langgraph.checkpoint.memory import MemorySaver

In [28]:
load_dotenv()

True

In [29]:
os.environ['GOOGLE_API_KEY'] = os.getenv('gemini_api_key') 
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [30]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]

In [31]:
def chat_node(state: ChatState): 

    messages = state['messages'] 

    response = model.invoke(messages) 

    return { "messages": [response]} 


In [42]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState) 

#Nodes 
graph.add_node('chat_node', chat_node) 

# Edges
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END) 

chatbot = graph.compile(checkpointer=checkpointer)

In [43]:
intial_state = {
    'messages': [HumanMessage(content='What is the Captial of India')]
}

In [44]:
chatbot.invoke(intial_state)['messages'][-1].content

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id

In [45]:
thread_id = '1'
while True:

    user_message = input('Type Here: ') 
    print('User: ', user_message)

    if user_message.strip() in ['exit', 'quit','bye']:
        break 
 
    config1 = { 'configurable': {'thread_id':"1"}}
    response = chatbot.invoke({'messages':[HumanMessage(content=user_message)]},config=config1) 

    print('AI: ', response['messages'][-1].content) 

User:  My name is saritha
AI:  Hi Saritha! It's nice to meet you.
User:  what is  my name?
AI:  Your name is Saritha!
User:  exit
